## First, we must import our dependancies 

#### Keras:  Building the actual network 
#### Numpy: is for loading in our data

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


First we load in the data that we have previously processed in a separate notebook

In [2]:
X_data = np.load('X_data.npy')
y_data = np.load('y_data.npy')
print('y shape: ', y_data.shape)
print('X shape: ', X_data.shape)

y shape:  (645, 1)
X shape:  (645, 400, 400)


Since the data is rather ordered, that is, all the y data is sequential, we must randomly split the data. We are going to make our training data large, since we do not have many images to feed our network to start with.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=55, train_size=0.95)

shape_1 = X_train.shape
shape_2 = X_test.shape # We will need the shapes for later resizing 

c:\users\ian\appdata\local\programs\python\python35\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


There is still a small amount of data preprocessing that needs to occur. Such as one hot encoding, and making the pixel values between 0-1.

In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32') #Converting the pixel values to floats

X_train /= 255
X_test /= 255 #This will make all the pixel values between 0 and 1

y_train = keras.utils.to_categorical(y_train, 8)
y_test = keras.utils.to_categorical(y_test, 8) # One-hot encoding

epochs = 2
batch_size = 150

X_train_fix = np.resize(X_train, (shape_1[0], shape_1[1], shape_1[2], 1))
X_test_fix = np.resize(X_test, (shape_2[0], shape_2[1], shape_2[2], 1))

In [5]:
X_train_fix[1:3].shape

(2, 400, 400, 1)

We can now begin to build the network. The following network has the same scaling factors as the Standford paper. 

In [6]:
model = Sequential()

#First layer 14x14x64
model.add(Conv2D(64, 14, input_shape=(400,400,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Activation('relu'))

#Second Layer 10x10x170
model.add(Conv2D(170, (10,10)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Activation('relu'))

#Third Layer 6x6x340
model.add(Conv2D(340, (6,6)))
model.add(Activation('relu'))

#Fourth Layer 6x6x340
model.add(Conv2D(340, (6,6)))
model.add(Activation('relu'))

#Fifth layer 6x6x340
model.add(Conv2D(340, (6,6)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Activation('relu'))

# Sixth layer: Fully connected 4048x1
model.add(Flatten())
model.add(Dense(4048))
model.add(Activation('relu'))
model.add(Dropout(0.25))

#Seventh Layer: Fully connected 4048x1
model.add(Dense(4048))
model.add(Activation('relu'))
model.add(Dropout(0.25))

#Eigth Layer: Fully connected 8x1
model.add(Dense(8, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 387, 387, 64)      12608     
_________________________________________________________________
activation_1 (Activation)    (None, 387, 387, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 129, 129, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 129, 129, 64)      0         
_________________________________________________________________
activation_2 (Activation)    (None, 129, 129, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 170)     1088170   
_________________________________________________________________
activation_3 (Activation)    (None, 120, 120, 170)     0         
__________

In [7]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(X_train_fix[1:3], y_train[1:3], batch_size=1, epochs=1)

Epoch 1/1
2/2 [==============================] - 27s - loss: 9.0971 - acc: 0.0000e+00     
